In [1]:
import numpy as np
import pandas as pd
import random
from random import randint

In [2]:
df = pd.read_csv('../resources/10.txt', delimiter=' ', engine='python')
parameters = df.columns.tolist()
df.columns = ['v', 'c']
df['w'] = df.index
data = df[['w','v','c']].values.tolist()
data = pd.DataFrame.from_records(data, columns=['w','v','c'])

data

,w,v,c
0,1329.0,1.2,179.0
1,443.0,1.0,260.0
2,407.0,0.6,188.0
3,921.0,0.8,259.0
4,1083.0,0.4,136.0
5,900.0,1.2,197.0
6,1332.0,0.5,206.0
7,562.0,0.9,291.0
8,1197.0,1.1,171.0
9,200.0,1.1,135.0


In [3]:
initialPopulation = 200 
numGenes = len(data) 
W = int(parameters[0]) 
V = int(parameters[1]) 
print('Лимиты:')
print('Вес:', W)
print('Объём:', V)

In [4]:
def checkW(individual):
    sumW = 0
    for i in range (0, numGenes):
        sumW += data.at[i,'w'] * individual[i]
    if sumW > W:
        return 0
    else: 
        return 1

def checkV(individual):
    sumV = 0
    for i in range (0, numGenes):
        sumV += data.at[i,'v'] * individual[i]
    if sumV > V:
        return 0
    else: 
        return 1

In [5]:
def createIndividual():
    individual = []
    for i in range(0, len(data)):
        individual.append(0)
    startIndex = randint(0, len(data) - 1)
    sumWeight = 0
    sumCapacity = 0
    for i in range(startIndex, len(data)):
        if (sumWeight + data.at[i,'w'] <= W) and (sumCapacity + data.at[i,'v'] <= V):
            individual[i] = 1
            sumWeight += data.at[i,'w']
            sumCapacity += data.at[i,'v']
    for i in range(0,startIndex):
        if (sumWeight + data.at[i,'w'] <= W) and (sumCapacity + data.at[i,'v'] <= V):
            individual[i] = 1
            sumWeight += data.at[i,'w']
            sumCapacity += data.at[i,'v']
    return individual

def createInitialPopulation():
    return [createIndividual() for _ in range(0, initialPopulation)]

In [6]:
def individualC(individual):
    sumC = 0
    for i in range(0, numGenes):
        sumC += data.at[i,'c'] * individual[i]
    return sumC

In [7]:
def selection(population):
    res_individuals = []
    individuals = [] 
    for i in range(len(population)):
        individuals.append(individualC(population[i]))
    
    max_fitness = max(individuals)
    min_fitness = min(individuals)

    for m in range(len(individuals)):
        if random.random() < individuals[m]/max_fitness:
            res_individuals.append(population[m])
    return res_individuals

In [8]:
def newChild(mother, father):
    child = []
    mask = [randint(0, 1) for x in range(numGenes)]
    for i in range(0, len(mask)):
        if mask[i] == 1:
            child.append(mother[i])
        else:
            child.append(father[i])
    return child

def crossingover(population):
    childs = []
    while len(population) > 0:
        random.shuffle(population)
        mother = population.pop()
        if len(population) == 0:
            break
        father = population.pop()
        child1 = newChild(mother, father)
        child2 = newChild(mother, father)
        if checkV(child1) and checkW(child1):
            childs.append(child1)
        if checkV(child2) and checkW(child2):
            childs.append(child2)
    return childs

In [9]:
def mutation(person):
    for i in range(0, len(person)):
        if person[i] == 0:
            person[i] = 1
        else:
            person[i] = 0
    return person

In [10]:
def GA():

    c = []
    C = 0
    for i in range (0, len(data)):
        c.append(data.at[i,'c'])
    for i in range (0, len(c)):
        C += c[i]
    
    minC = min(c)
    oldC = 0
    newC = C
    maxIteration = 500

    # Начальная популяция
    population = createInitialPopulation()

    while maxIteration > 0:
        if maxIteration % 10 == 0:
            print(maxIteration / 10)

        # Отбор
        population = selection(population)

        # Скрещивание
        childs = crossingover(population.copy())

        # Мутация
        rndIndividualInd = randint(0, len(population) - 1)
        population[rndIndividualInd] = mutation(population[rndIndividualInd])

        # Новая популяция
        for individual in population:
            individual.append(individualC(individual) * 0.9)
        for child in childs:
            child.append(individualC(child))
        population.extend(childs)
        population = list(filter(lambda x: checkW(x) and checkV(x), population))
        population.sort(key=lambda x: individualC(x), reverse=True)
        population = population[0:200]
        bestIndividual = population[0]

        maxIteration -= 1
    return bestIndividual

In [11]:
bestIndividual = GA()

50.0
49.0
48.0
47.0
46.0
45.0
44.0
43.0
42.0
41.0
40.0
39.0
38.0
37.0
36.0
35.0
34.0
33.0
32.0
31.0
30.0
29.0
28.0
27.0
26.0
25.0
24.0
23.0
22.0
21.0
20.0
19.0
18.0
17.0
16.0
15.0
14.0
13.0
12.0
11.0
10.0
9.0
8.0
7.0
6.0
5.0
4.0
3.0
2.0
1.0


In [12]:
sumW, sumV, sumC = 0, 0, 0
result = []
for i in range(numGenes):
    sumW += bestIndividual[i] * data.at[i,'w']
    sumV += bestIndividual[i] * data.at[i,'v']
    sumC += bestIndividual[i] * data.at[i,'c']
    if bestIndividual[i] == 1:
        result.append([data.at[i,'w'], data.at[i,'v'], data.at[i,'c']])
result_df = pd.DataFrame.from_records(result, columns=['w','v','c'])

In [13]:
print(result_df)
print("Вес:", sumW)
print("Объем:", round(sumV,1))
print("Ценность:", sumC)

         w    v      c
0    443.0  1.0  260.0
1    407.0  0.6  188.0
2    921.0  0.8  259.0
3    562.0  0.9  291.0
4   1214.0  0.7  301.0
5    976.0  0.6  131.0
6   1344.0  0.9  309.0
7    150.0  0.7  259.0
8    453.0  0.9  273.0
9    856.0  0.9  300.0
10   725.0  0.9  258.0
11  1044.0  0.8  261.0
12   288.0  0.6  286.0
13   320.0  0.4  142.0
14  1664.0  0.7  262.0
15  1545.0  0.6  254.0
Вес: 12912.0
Объем: 12.0
Ценность: 4034.0


In [15]:
file=open('kovshov_hw4_2_res.txt','w', encoding='utf-8')
file.write(result_df.to_string())
file.write("\nВес: " + str(sumW))
file.write("\nОбъем: " + str(round(sumV,1)))
file.write("\nЦенность: " + str(sumC))
file.close()